In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import pandas as pd
import scanpy as sc
import anndata as ad
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import adjusted_rand_score
import community
import networkx as nx
import gzip
from scipy.sparse import csc_matrix, csr_matrix
import scipy.sparse
from scipy.io import mmwrite
import scib
from scipy import stats

import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

%matplotlib inline

KeyboardInterrupt: 

In [ ]:

###############
dataset="human_pancreas_norm_complexBatch"
###############

data_dir="/goofys/users/Aleksandra_S/benchmarking_datasets/"+dataset+".h5ad"
adata=ad.read_h5ad(data_dir)

/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/specs/registry.py:235: OldFormatWarning: Element '/X' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/specs/registry.py:235: OldFormatWarning: Element '/layers' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/specs/registry.py:235: OldFormatWarning: Element '/layers/counts' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/obs/__categories/tech' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/

In [ ]:
adata.obs['batch'] = adata.obs.tech

In [ ]:
unique_batches = adata.obs['batch'].unique()
# unpack anndata (divide into batches)
# Initialize a dictionary to store the split AnnData objects
split_adatas = {}

# Split the original AnnData object into separate batches
for batch in unique_batches:
    mask = adata_raw.obs['batch'] == batch
    split_adatas[batch] = adata_raw[mask].copy()
adata_list =  [adata for adata in split_adatas.values()]
#split_adatas

In [ ]:
unique_batches

['celseq', 'celseq2', 'fluidigmc1', 'smartseq2', 'inDrop1', 'inDrop2', 'inDrop3', 'inDrop4', 'smarter']
Categories (9, object): ['celseq', 'celseq2', 'fluidigmc1', 'inDrop1', ..., 'inDrop3', 'inDrop4', 'smarter', 'smartseq2']

In [ ]:
list_index = 8 # if == "All", it means there will be no slicing by cluster
slice_name = 'c5'
if list_index == 'All':
    batch_name = ""
else:
    batch_name = unique_batches[list_index]

### Slicing Cluster (optional)

In [ ]:
if list_index != 'All':
    adata = adata_list[list_index].copy()

## Highly Deviant Genes
racuna se nad RAW podacima! <br>
quantifies whether genes show a constant expression profile across cells as these are not informative.  <br>
Genes with constant expression are described by a multinomial null model, they are approximated by the binomial deviance.  <br>
Highly informative genes across cells will have a high deviance value which indicates a poor fit by According to the deviance values, the method then ranks all genes and obtains only highly deviant genes. <br>

In [ ]:
%%R
library(scry)

In [ ]:
raw_adata = ad.AnnData(X=adata.layers['counts'])

In [ ]:
ro.globalenv["adata"] = raw_adata

In [ ]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [ ]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T

In [ ]:
idx = binomial_deviance.argsort()[-5000:]
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[idx] = True

adata.var["highly_deviant"] = mask
adata.var["binomial_deviance"] = binomial_deviance

In [ ]:
highly_deviant_df = adata.var[adata.var.highly_deviant].sort_values(by='binomial_deviance', ascending=False)
highly_deviant_names = highly_deviant_df.index.values

In [ ]:
highly_deviant_df.to_csv(f"/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/highly_deviant_{batch_name}.csv")

# Merging

In [2]:
highly_deviant_celseq = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_celseq.csv")
highly_deviant_celseq2 = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_celseq2.csv")
highly_deviant_fluidigmc1 = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_fluidigmc1.csv")
highly_deviant_inDrop1 = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_inDrop1.csv")
highly_deviant_inDrop2 = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_inDrop2.csv")
highly_deviant_inDrop3 = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_inDrop3.csv")
highly_deviant_inDrop4 = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_inDrop4.csv")
highly_deviant_smarter = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_smarter.csv")
highly_deviant_smartseq2 = pd.read_csv("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/highly_deviant_smartseq2.csv")

In [3]:
highly_deviant_celseq["Unnamed: 0"].values[:2000]

array(['GCG', 'SST', 'REG1A', ..., 'C14orf2', 'SUCLG1', 'NDUFB6'],
      dtype=object)

In [4]:
highly_deviant_celseq_hvgs = highly_deviant_celseq["Unnamed: 0"].values[:2000] 
highly_deviant_celseq2_hvgs = highly_deviant_celseq2["Unnamed: 0"].values[:2000]
highly_deviant_fluidigmc1_hvgs = highly_deviant_fluidigmc1["Unnamed: 0"].values[:2000]
highly_deviant_inDrop1_hvgs = highly_deviant_inDrop1["Unnamed: 0"].values[:2000]
highly_deviant_inDrop2_hvgs = highly_deviant_inDrop2["Unnamed: 0"].values[:2000]
highly_deviant_inDrop3_hvgs = highly_deviant_inDrop3["Unnamed: 0"].values[:2000]
highly_deviant_inDrop4_hvgs = highly_deviant_inDrop4["Unnamed: 0"].values[:2000]
highly_deviant_smarter_hvgs = highly_deviant_smarter["Unnamed: 0"].values[:2000]
highly_deviant_smartseq2_hvgs = highly_deviant_smartseq2["Unnamed: 0"].values[:2000]

In [5]:
highly_deviant_celseq_hvgs

array(['GCG', 'SST', 'REG1A', ..., 'C14orf2', 'SUCLG1', 'NDUFB6'],
      dtype=object)

In [6]:
hdgs_union = set(highly_deviant_celseq_hvgs).union(highly_deviant_celseq2_hvgs, highly_deviant_fluidigmc1_hvgs, highly_deviant_inDrop1_hvgs, highly_deviant_inDrop2_hvgs, highly_deviant_inDrop3_hvgs, highly_deviant_inDrop4_hvgs, highly_deviant_smarter_hvgs, highly_deviant_smartseq2_hvgs)

In [7]:
batches = [highly_deviant_celseq_hvgs, highly_deviant_celseq2_hvgs, highly_deviant_fluidigmc1_hvgs, highly_deviant_inDrop1_hvgs, highly_deviant_inDrop2_hvgs, highly_deviant_inDrop3_hvgs, highly_deviant_inDrop4_hvgs, highly_deviant_smarter_hvgs, highly_deviant_smartseq2_hvgs]

In [8]:
hdg_in_batch = np.zeros(shape=len(hdgs_union))
for i, hdg in enumerate(hdgs_union):
    for batch in batches:
        if hdg in batch:
            hdg_in_batch[i]+=np.where(batch == hdg)[0][0]



In [9]:
top_2000_indices = np.argsort(hdg_in_batch)[::-1][:2000]

In [10]:
final_2000_hdgs = [list(hdgs_union)[i] for i in top_2000_indices]

In [11]:
final_2000_hdgs_arr = np.array(list(hdgs_union))[top_2000_indices]

In [12]:
final_2000_hdgs_arr

array(['CCT2', 'LMAN1', 'GNG12', ..., 'FASN', 'CCDC105', 'C1orf189'],
      dtype='<U11')

In [14]:
# final_2000_hdgs

In [18]:
np.save("/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/pancreas/final_2000_hdgs.npy", final_2000_hdgs_arr)